In [1]:
#2
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [3]:
from typing import List, Dict, TypedDict, Optional
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
import re

# 카페 메뉴 데이터 로드 (로컬 경로 사용)
menu_path = "C:\\mylangchain\\langchain_basic\\data\\cafe_menu.txt"
with open(menu_path, "r", encoding="utf-8") as f:
    menu_data = f.read()

# 문서 분할 및 벡터 DB 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.create_documents([menu_data])
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
menu_db = FAISS.from_documents(docs, embeddings)

# 메뉴 메타데이터 추가
for doc in menu_db.docstore._dict.values():
    if "메뉴명:" in doc.page_content:
        doc.metadata["menu_name"] = doc.page_content.split("메뉴명:")[1].split("\n")[0].strip()

C:\Users\user\AppData\Local\Temp\ipykernel_21400\3956760135.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.13\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `hugg

In [4]:
class ChatState(TypedDict):
    messages: List[HumanMessage | AIMessage]
    inquiry_type: Optional[str]
    retrieved_docs: Optional[List[Document]]

def classify_inquiry(state: ChatState) -> ChatState:
    user_message = state["messages"][-1].content.lower()
    
    if any(keyword in user_message for keyword in ["가격", "얼마", "비용"]):
        return {**state, "inquiry_type": "price"}
    elif any(keyword in user_message for keyword in ["추천", "어떤 게 좋", "뭐가 좋"]):
        return {**state, "inquiry_type": "recommendation"}
    else:
        return {**state, "inquiry_type": "menu"}

def retrieve_menu_info(state: ChatState) -> ChatState:
    user_message = state["messages"][-1].content
    
    if state["inquiry_type"] == "price":
        docs = menu_db.similarity_search("메뉴 가격", k=5)
    elif state["inquiry_type"] == "recommendation":
        docs = menu_db.similarity_search(user_message, k=3)
        if not docs:
            docs = menu_db.similarity_search("인기 메뉴", k=3)
    else:
        docs = menu_db.similarity_search(user_message, k=4)
    
    return {**state, "retrieved_docs": docs}

def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', content.split("\n")[0].replace("메뉴명:", "").strip())
    
    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    category_match = re.search(r'카테고리:\s*(.+?)(?:\n|$)', content)
    
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음",
        "category": category_match.group(1).strip() if category_match else "카테고리 정보 없음"
    }

def generate_response(state: ChatState) -> ChatState:
    user_message = state["messages"][-1]
    inquiry_type = state["inquiry_type"]
    docs = state["retrieved_docs"]
    
    if not docs:
        response = "죄송합니다. 해당하는 메뉴 정보를 찾을 수 없습니다."
    else:
        menu_infos = [extract_menu_info(doc) for doc in docs]
        
        if inquiry_type == "price":
            price_list = "\n".join([f"- {info['name']}: {info['price']}" for info in menu_infos])
            response = f"다음은 메뉴별 가격 정보입니다:\n{price_list}"
        elif inquiry_type == "recommendation":
            recommended = menu_infos[0]
            response = f"추천 메뉴: {recommended['name']}\n"
            response += f"가격: {recommended['price']}\n"
            response += f"설명: {recommended['description']}\n"
            response += "이 메뉴는 어떠신가요?"
        else:
            if len(menu_infos) == 1:
                info = menu_infos[0]
                response = f"{info['name']}에 대한 정보입니다:\n"
                response += f"가격: {info['price']}\n"
                response += f"설명: {info['description']}\n"
                response += f"카테고리: {info['category']}"
            else:
                menu_names = "\n".join([f"- {info['name']}" for info in menu_infos])
                response = f"다음과 관련된 메뉴들을 찾았습니다:\n{menu_names}\n"
                response += "더 자세한 정보를 원하시면 메뉴 이름을 정확히 알려주세요."
    
    return {**state, "messages": [*state["messages"], AIMessage(content=response)]}

In [18]:
# 그래프 구성
workflow = MessageGraph()

# 노드 추가
workflow.add_node("classify_inquiry", classify_inquiry)
workflow.add_node("retrieve_info", retrieve_menu_info)
workflow.add_node("generate_response", generate_response)

# 엣지 설정
workflow.add_edge("classify_inquiry", "retrieve_info")
workflow.add_edge("retrieve_info", "generate_response")
workflow.add_edge("generate_response", END)

# 진입점 설정
workflow.set_entry_point("classify_inquiry")

# 그래프 컴파일
app = workflow.compile()

In [20]:
# 5. 테스트 실행 함수
def test_agent(question: str):
    print(f"\n[질문] {question}")
    state = {
        "messages": [HumanMessage(content=question)],
        "inquiry_type": None,
        "retrieved_docs": None
    }
    result = app.invoke(state)
    for msg in result["messages"]:
        if isinstance(msg, AIMessage):
            print(f"[응답] {msg.content}")

# 6. 실제 테스트
test_agent("아이스 아메리카노 가격이 얼마인가요?")
test_agent("추천해주실 만한 커피 메뉴가 있나요?")
test_agent("카페라떼에 대해 알려주세요")
test_agent("가장 인기 있는 음료는 뭔가요?")


[질문] 아이스 아메리카노 가격이 얼마인가요?


ValueError: Message dict must contain 'role' and 'content' keys, got {'messages': [HumanMessage(content='아이스 아메리카노 가격이 얼마인가요?', additional_kwargs={}, response_metadata={})], 'inquiry_type': None, 'retrieved_docs': None}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 

In [ ]:
test_cases = [
    "아이스 아메리카노 가격 알려줘",
    "딸기 스무디 맛있나요?",
    "오늘 추천 메뉴 뭐야?",
    "카페라떼에 대해 알려주세요",
    "가장 인기 있는 음료는 뭔가요?"
]

for query in test_cases:
    print(f"\n사용자 질문: {query}")
    result = app.invoke({"messages": [HumanMessage(content=query)]})
    print(f"AI 응답: {result['messages'][-1].content}")

In [21]:
from typing import TypedDict, List, Optional
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
import re

# 1. 상태 정의 (수정된 버전)
class AgentState(TypedDict):
    messages: List[dict]  # dict 형태로 저장
    inquiry_type: Optional[str]
    retrieved_docs: Optional[List[Document]]

# 2. 카페 메뉴 데이터 로드 및 벡터 DB 생성
def initialize_menu_db():
    menu_path = "C:\\mylangchain\\langchain_basic\\data\\cafe_menu.txt"
    with open(menu_path, "r", encoding="utf-8") as f:
        menu_data = f.read()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.create_documents([menu_data])
    embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
    menu_db = FAISS.from_documents(docs, embeddings)
    
    for doc in menu_db.docstore._dict.values():
        if "메뉴명:" in doc.page_content:
            doc.metadata["menu_name"] = doc.page_content.split("메뉴명:")[1].split("\n")[0].strip()
    return menu_db

menu_db = initialize_menu_db()

# 3. 메시지 변환 도우미 함수
def to_dict(message: HumanMessage | AIMessage) -> dict:
    return {
        "role": "human" if isinstance(message, HumanMessage) else "ai",
        "content": message.content
    }

def from_dict(message_dict: dict) -> HumanMessage | AIMessage:
    if message_dict["role"] == "human":
        return HumanMessage(content=message_dict["content"])
    return AIMessage(content=message_dict["content"])

# 4. 핵심 기능 함수들
def classify_inquiry(state: AgentState) -> AgentState:
    last_msg = from_dict(state["messages"][-1])
    content = last_msg.content.lower()
    
    if any(k in content for k in ["가격", "얼마", "비용"]):
        return {**state, "inquiry_type": "price"}
    elif any(k in content for k in ["추천", "어떤 게 좋", "뭐가 좋"]):
        return {**state, "inquiry_type": "recommendation"}
    else:
        return {**state, "inquiry_type": "menu"}

def retrieve_menu_info(state: AgentState) -> AgentState:
    last_msg = from_dict(state["messages"][-1])
    query = last_msg.content
    
    if state["inquiry_type"] == "price":
        docs = menu_db.similarity_search("메뉴 가격", k=5)
    elif state["inquiry_type"] == "recommendation":
        docs = menu_db.similarity_search(query, k=3)
        docs = docs or menu_db.similarity_search("인기 메뉴", k=3)
    else:
        docs = menu_db.similarity_search(query, k=4)
    
    return {**state, "retrieved_docs": docs}

def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    return {
        "name": doc.metadata.get("menu_name", "Unknown"),
        "price": re.search(r'₩([\d,]+)', content).group(0) if re.search(r'₩([\d,]+)', content) else "가격 정보 없음",
        "description": re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL).group(1).strip() 
                   if re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL) else "설명 없음"
    }

def generate_response(state: AgentState) -> AgentState:
    if not state["retrieved_docs"]:
        response = "죄송합니다. 해당 메뉴 정보를 찾을 수 없습니다."
    else:
        menus = [extract_menu_info(doc) for doc in state["retrieved_docs"]]
        
        if state["inquiry_type"] == "price":
            response = "메뉴 가격 정보:\n" + "\n".join([f"- {m['name']}: {m['price']}" for m in menus])
        elif state["inquiry_type"] == "recommendation":
            rec = menus[0]
            response = f"추천 메뉴: {rec['name']}\n가격: {rec['price']}\n설명: {rec['description']}"
        else:
            response = "관련 메뉴:\n" + "\n".join([f"- {m['name']} ({m['price']})" for m in menus])
    
    new_messages = state["messages"] + [{"role": "ai", "content": response}]
    return {**state, "messages": new_messages}

# 5. 그래프 구성
workflow = StateGraph(AgentState)
workflow.add_node("classify", classify_inquiry)
workflow.add_node("retrieve", retrieve_menu_info)
workflow.add_node("respond", generate_response)

workflow.set_entry_point("classify")
workflow.add_edge("classify", "retrieve")
workflow.add_edge("retrieve", "respond")
workflow.add_edge("respond", END)

app = workflow.compile()

# 6. 테스트 실행
def run_test(question: str):
    print(f"\n[질문] {question}")
    state = {
        "messages": [{"role": "human", "content": question}],
        "inquiry_type": None,
        "retrieved_docs": None
    }
    result = app.invoke(state)
    for msg in result["messages"]:
        print(f"[{msg['role']}] {msg['content']}")

# 7. 테스트 실행
run_test("아이스 아메리카노 가격이 얼마인가요?")
run_test("추천해주실 만한 커피 메뉴가 있나요?")
run_test("카페라떼에 대해 알려주세요")


[질문] 아이스 아메리카노 가격이 얼마인가요?
[human] 아이스 아메리카노 가격이 얼마인가요?
[ai] 메뉴 가격 정보:
- Unknown: 가격 정보 없음
- Unknown: 가격 정보 없음
- Unknown: 가격 정보 없음
- Unknown: 가격 정보 없음
- Unknown: 가격 정보 없음

[질문] 추천해주실 만한 커피 메뉴가 있나요?
[human] 추천해주실 만한 커피 메뉴가 있나요?
[ai] 추천 메뉴: Unknown
가격: 가격 정보 없음
설명: 진한 에스프레소와 뜨거운 물의 조화로 깔끔한 맛

[질문] 카페라떼에 대해 알려주세요
[human] 카페라떼에 대해 알려주세요
[ai] 관련 메뉴:
- Unknown (가격 정보 없음)
- Unknown (가격 정보 없음)
- Unknown (가격 정보 없음)
- Unknown (가격 정보 없음)
